In [1]:
import data_utils

dataset = data_utils.get_dataset()

In [2]:
dataset = data_utils.clean_afec_dpto(dataset)
dataset = data_utils.clean_riesgo_vida(dataset)
dataset = data_utils.clean_cie_10(dataset)
dataset = data_utils.remove_features(dataset)
dataset = dataset.reset_index()
dataset = dataset.drop(['index'], axis = 1)

/Users/mrugeles/Documents/DataScience/Supersalud-PQRSD/data_utils.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  dataset['RIESGO_VIDA'] = np.where(dataset['RIESGO_VIDA'] == 'si', 1,0)


In [3]:
zero_values = set(dataset.columns[dataset.eq('0').mean() > 0])
for feature in zero_values:
    dataset[f'{feature}_is_missing'] = dataset[feature].apply(lambda f: 1 if f == '0' else 0)

features_columns = [column for column in dataset.columns if '_is_missing' not in column]
missing_state_columns = [column for column in dataset.columns if '_is_missing' in column]

dataset[missing_state_columns].head()

,ENT_AMBITOIVC_is_missing,AFEC_GENERO_is_missing,ENT_MPIO_DANE_is_missing,PATOLOGIA_TIPO_is_missing,ENT_COD_DEPTO_is_missing,ENT_COD_MPIO_is_missing,ENT_TIPOVIG_is_missing,PET_MPIO_is_missing,IDPATOLOGIA_2_is_missing,PATOLOGIA_1_is_missing,...,AFEC_TIPOPER_is_missing,ALTO_COSTO_is_missing,ENT_COD_SNS_is_missing,AFEC_DPTO_is_missing,PET_TIPOPER_is_missing,AFEC_EDADR_is_missing,AFEC_EDUC_is_missing,ENT_NOMBRE_is_missing,PQR_TIPOATENCION_is_missing,AFEC_MPIO_is_missing
0,0,0,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,1,0
2,1,0,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,1,0
3,1,0,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0


## Encoding dataset

In [4]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
for column in features_columns:
    le.fit(dataset[column])
    dataset[column] = le.transform(dataset[column]) 

dataset.head()

,AFEC_DPTO,AFEC_EDADR,AFEC_EDUC,AFEC_GENERO,AFEC_GETNICO,AFEC_MPIO,AFEC_PARENTESCO,AFEC_POBESPECIAL,AFEC_REGAFILIACION,AFEC_TIPOPER,...,AFEC_TIPOPER_is_missing,ALTO_COSTO_is_missing,ENT_COD_SNS_is_missing,AFEC_DPTO_is_missing,PET_TIPOPER_is_missing,AFEC_EDADR_is_missing,AFEC_EDUC_is_missing,ENT_NOMBRE_is_missing,PQR_TIPOATENCION_is_missing,AFEC_MPIO_is_missing
0,5,2,1,2,5,123,12,4,5,2,...,0,1,0,0,0,0,0,0,0,0
1,31,2,5,1,5,161,1,4,1,2,...,0,1,0,0,0,0,0,0,1,0
2,18,2,8,1,5,676,13,4,1,2,...,0,1,0,0,0,0,0,0,1,0
3,26,2,1,1,5,322,13,5,1,2,...,0,1,0,0,0,0,0,0,1,0
4,5,2,1,1,5,123,12,4,5,2,...,0,1,0,0,0,0,0,0,0,0


## Store processed dataset

In [5]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split


dataset.to_csv("datasets/missing_state_dataset.csv", index = False)